### ReAct
It's an agents pattern for LLMs where they are provided with a set of tools. Based on incoming questions, they reason to select the appropriate tool and provide it with relevant input to generate the required answer.
Here, function incocation is not the responsiblity of the developer but rather the LLM.

See [here](https://react-lm.github.io/)

We take forward the same problem as discussed in the [Here](https://github.com/afraz-khan/gen-ai/blob/main/Agents/Function-Calling/Ollama%20%7C%20Local-Model.ipynb)

#### Langchain wrapper
LangChain has developed a standard API called _`create_react_agent`_ for building ReAct agents. It's essential to test and verify if your model supports this API before implementation. I used multiple local LLMs but [Claude-3.5-Sonnet](https://www.anthropic.com/news/claude-3-5-sonnet) testing was ranked top.

#### Install Deps

In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain import hub
from langchain.tools.render import render_text_description
from langchain_community.llms import Ollama
from langchain.pydantic_v1 import BaseModel, Field
from IPython.display import display, Markdown, Latex
from langchain_aws import ChatBedrock

### Setup the model

In [ ]:
llm = ChatOpenAI(
    model="RichardErkhov/openai-community_-_gpt2-xl-gguf",
    temperature=0, api_key="lm-studio",
    base_url="http://localhost:1234/v1"
)

llm = Ollama(model="llama3", format='json')

model_kwargs = {
    "temperature": 0,
    "max_tokens": 4096,
    "anthropic_version": "bedrock-2023-05-31",
    "top_p": 1,
    "top_k": 50
}
llm = ChatBedrock(
    model_id='anthropic.claude-3-5-sonnet-20240620-v1:0',
    model_kwargs=model_kwargs,
    region_name='us-east-1',
    credentials_profile_name='default'
)

### Define Tools and Calling Service

In [ ]:
class PhoneNumberStatusCheckInput(BaseModel):
    phone_number: str = Field(description="should be a phone number string")

class PhoneNumberReactivatorInput(BaseModel):
    phone_number: str = Field(description="should be a phone number string")

class FinalMessageInput(BaseModel):
    last_action_name: str = Field(description="should be the name of last executed action")
    last_action_output: dict = Field(default_factory=dict, description="should be python dictionary based the output from last executed action")

@tool("check-phone-number-status", args_schema=PhoneNumberStatusCheckInput)
def check_phone_number_status(phone_number: str):
    """Useful when you want to check the status of a phone number"""
    phone_statuses = {"1234567890": True, "0987654321": False}
    status = phone_statuses.get(phone_number, False)
    return {"active": status}

@tool("request-phone-number-reactivation", args_schema=PhoneNumberReactivatorInput)
def request_phone_number_reactivation(phone_number: str):
    """Useful when you want to submit a request to reactivate a phone number."""
    return {"request_submitted": True}

@tool("prepare-final-message", args_schema=FinalMessageInput, return_direct=True)
def prepare_final_message(last_action_name: str, last_action_output: dict):
    """Useful when you want to create the final message for the user. Pass it arguments like {{"last_action_name": "some-tool", "last_action_output": {"..key": "..val"}}}"""

    print(last_action_name, last_action_output)

    if last_action_name == 'check-phone-number-status':
        output = last_action_output['active']
        if not output: return "Your phone number is currently inactive. Do you want to reactivate it?"
        return "Phone number is currently active."

    if last_action_name == 'request-phone-number-reactivation':
        output = last_action_output['request_submitted']
        if not output: return "Your request is submitted successfully."
        return "Cant submit your request."
    
    return {"request_submitted": True}

tools = [check_phone_number_status, request_phone_number_reactivation]

I encountered a challenge with the `prepare_final_message` tool; the LLM didn't call it correctly. It seems that additional testing and fine-tuning are needed. However, ReAct functioned correctly with the other two tools and the use case questions

### Setup the Agent

In [ ]:
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)

### Invoke Agent

In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

# question = "check phone number status: 1234567890"
# question = "check phone number status: 0987654321"
question = "Please reactivate the phone number 0987654321"

response = agent_executor.invoke({"input": question, "agent_scratchpad" : [], "chat_history": []})

In [ ]:
display(Markdown(response["output"]))